<a href="https://colab.research.google.com/github/SmittyB00p/PhiuSFBv0NkHjLd3/blob/main/fine_tuning_Phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Implementation

----------------------------------- Note ------------------------------------

If using this notebook in its entirety, you will need a GPU and at least 25GB of working memory.

-------------------------------------_---------------------------------------

* The top portion of this notebook implements Retrieval-Augmented Generation (RAG).
* The bottom portion fine-tunes the ```microsoft/Phi-3-mini-4k-instruct``` model from HuggingFace with the help of ```unsloth``` to make the process go 2X faster.

Note at the bottom of the notebook how when I asked it the follow up question of ```'Why did you chose the first candidate when there were other candidates that had the same job title?'``` it responded with:

```The selection was based on the specific requirements and preferences outlined in the query. The first candidate was chosen because they had a relevant educational background and were located in the desired area.```

And then when I asked it to define the 'desired area' it responds:

```The desired area refers to the geographical location where an individual is seeking employment opportunities. It is the specific region or city where they are interested in working and relocating to.```

You can see that this way of ranking candidates does have its biases. In this case it is a geographical bias. Unfortunately, I was unable to continue the conversation because of runtime issues. If I could have continued, I was going to ask it how it would rank the candidates if I was looking for candidates in a different region, but alas the computational excesses did not allow for the conversation to continue.



In [10]:
!pip install trl
!pip install -U bitsandbytes
# !pip install datasets
!pip install deepspeed
!pip install llama-index
!pip install flash_attn
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install peft
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import huggingface_hub, torch
# !pip install -U transformers==4.31.0
import transformers
import trl
import datasets
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, RagTokenizer, RagRetriever, RagSequenceForGeneration
from trl import SFTTrainer
import peft
import bitsandbytes
import sys, logging
from peft import LoraConfig


  Using cached flash_attn-2.7.3.tar.gz (3.2 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 6.6 MB/s eta 0:00:00


In [11]:
from trl import SFTTrainer
import llama_index
from llama_index.llms.openai import OpenAI

In [12]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = 'expandable_segments:True'

from google.colab import drive
drive = drive.mount('/content/drive')
from google.colab import userdata
user_data = userdata.get('phi-3_fine_tuning_key') ## enter your own api key from hugging face

cwd = os.getcwd()
df = pd.read_csv(cwd+'/drive/MyDrive/Data2/pre-ranked_df.csv')
candidates = df['job_title'].tolist()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
logger = logging.getLogger(__name__)


###################
# Hyper-parameters
###################
training_config = {
    "bf16": False,
    "do_eval": False,
    "learning_rate": 5.0e-06,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 1,
    "max_steps": -1,
    "output_dir": "/content/MyDrive/Phi-3FineTuning/checkpoint_dir",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 2,
    "per_device_train_batch_size": 2,
    "remove_unused_columns": True,
    "save_steps": 60,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.2,
    # "deepspeed": "/content/drive/MyDrive/Phi-3FineTuning/deepspeed.json",
    }

peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": "all-linear",
    "modules_to_save": None,
}

if torch.cuda.is_bf16_supported():
    compute_dtype = torch.bfloat16
else:
    compute_dtype = torch.float16

train_conf = TrainingArguments(**training_config)
peft_conf = LoraConfig(**peft_config)

In [14]:
###############
# Setup logging
###############
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = train_conf.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process a small summary
logger.warning(
    f"Process rank: {train_conf.local_rank}, device: {train_conf.device}, n_gpu: {train_conf.n_gpu}"
    + f" distributed training: {bool(train_conf.local_rank != -1)}, 16-bits training: {train_conf.fp16}"
)
logger.info(f"Training/evaluation parameters {train_conf}")
logger.info(f"PEFT parameters {peft_conf}")

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=Non

In [15]:
# ################
# # Model Loading
# ################
checkpoint_path = "microsoft/Phi-3-mini-4k-instruct"
# # checkpoint_path = "microsoft/Phi-3-mini-128k-instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    attn_implementation="eager",  # loading the model with flash-attenstion support
    torch_dtype=torch.bfloat16,
    device_map=None
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 2048
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

[INFO|configuration_utils.py:695] 2025-01-16 19:27:35,398 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json


configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

[WARNING|dynamic_module_utils.py:422] 2025-01-16 19:27:35,538 >> A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
[INFO|configuration_utils.py:695] 2025-01-16 19:27:35,542 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
[INFO|configuration_utils.py:762] 2025-01-16 19:27:35,543 >> Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-

modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:30:47,406 >> loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/tokenizer.model
[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:30:47,407 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/tokenizer.json
[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:30:47,407 >> loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/added_tokens.json
[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:30:47,408 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/special_tokens_map

In [ ]:
## after you have loaded the model and tokenizer you can save them as to not have to not load them in again
## you can then comment out the last 6 lines above
## after you have saved the model and tokenizer you can then comment these lines out and uncomment the lines in the next code block

model.save_pretrained(cwd+'/drive/MyDrive/phi3')
tokenizer.save_pretrained(cwd+'/drive/MyDrive/phi3')

In [ ]:
# model = AutoModelForCausalLM.from_pretrained(cwd+'/drive/MyDrive/phi3', **model_kwargs)
# tokenizer = AutoTokenizer.from_pretrained(cwd+'/drive/MyDrive/phi3')

In [16]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

In [17]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, SummaryIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

def messages_to_prompt(messages):
  prompt = ''
  system_found = False
  for message in messages:
    if message.role == 'system':
      prompt += f'<|system|>\n{message.content}<|end|>\n'
      system_found = True
    elif message.role == 'user':
      prompt += f'<|user|>\n{message.content}<|end|>\n'
    elif message.role == 'assistant':
      prompt += f'<|assistant|>\n{message.content}<|end|>\n'
    else:
      prompt += f'<|user|>\n{message.content}<|end|>\n'

  prompt += '<|assistant|>\n'

  if not system_found:
    prompt = (
        '<|system|>\nYou are a helpful AI assistant that answers questions to the best of your knowledge base.<|end|>' + prompt
    )

  return prompt

llm = HuggingFaceLLM(
    # disk_offload,
    model_name="microsoft/Phi-3-mini-4k-instruct",
    tokenizer=tokenizer,
    model_kwargs={'trust_remote_code':True},
    generate_kwargs={'do_sample': True, 'temperature': 0.1},
    query_wrapper_prompt=(
        '<|system|>\n'
        'You are a helpful AI assistant that answers questions to the best of your knowledge base.<|end|>\n'
        '<|user|>\n'
        '{query_str}<|end|>\n'
        '<|assistant|>\n'
    ),
    messages_to_prompt=messages_to_prompt,
    is_chat_model=True
)

[INFO|configuration_utils.py:695] 2025-01-16 19:30:55,874 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
[INFO|configuration_utils.py:695] 2025-01-16 19:30:55,921 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
[INFO|configuration_utils.py:762] 2025-01-16 19:30:55,923 >> Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id":

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4849] 2025-01-16 19:31:11,339 >> All model checkpoint weights were used when initializing Phi3ForCausalLM.

[INFO|modeling_utils.py:4857] 2025-01-16 19:31:11,341 >> All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
[INFO|configuration_utils.py:1095] 2025-01-16 19:31:11,388 >> loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/generation_config.json
[INFO|configuration_utils.py:1140] 2025-01-16 19:31:11,390 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}



In [18]:
Settings.llm = llm

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:31:13,823 >> loading file vocab.txt from cache at /tmp/llama_index/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a/vocab.txt
[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:31:13,824 >> loading file tokenizer.json from cache at /tmp/llama_index/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a/tokenizer.json
[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:31:13,825 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:31:13,825 >> loading file special_tokens_map.json from cache at /tmp/llama_index/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a/special_tokens_map.json
[INFO|tokenization_utils_base.py:2030] 2025-01-16 19:31:13,826 >> loading file tokenizer_config.json from cache at /tmp/llama_index/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
# from llama_index import node_parser

# node_parser = SimpleNodeParser(chunk_size_limit=512)
documents = SimpleDirectoryReader("drive/MyDrive/Data").load_data()
documents

[Document(id_='a346f304-6dd7-46b6-b2e3-8577d216ecbb', embedding=None, metadata={'file_path': '/content/drive/MyDrive/Data/potential-talents - Aspiring human resources - seeking human resources.csv', 'file_name': 'potential-talents - Aspiring human resources - seeking human resources.csv', 'file_type': 'text/csv', 'file_size': 9314, 'creation_date': '2025-01-12', 'last_modified_date': '2025-01-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="1, 2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional, Houston, Texas, 85, nan\n2, Native English Teacher at EPIK (English Program in Korea), K

In [20]:
vector_index = VectorStoreIndex.from_documents(documents, chunk_size_limit=512)

summary_index = SummaryIndex.from_documents(documents)
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [21]:
print(os.environ['PYTORCH_CUDA_ALLOC_CONF'])

expandable_segments:True


In [24]:
import gc
gc.collect()

query_engine = vector_index.as_query_engine(response_mode='compact')

response = query_engine.query("Can you give me a list of the top 15 candidates that have 'aspiring human resources' in their title? As well as the reason you chose each candidate")

In [25]:
from llama_index.core.response.notebook_utils import display_response
display_response(response)

**`Final Response:`** 1. 2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional, Houston, Texas, 85, nan
   - This candidate has a graduate degree from a prestigious business school and is actively seeking a career in Human Resources.

2. Native English Teacher at EPIK (English Program in Korea), Kanada, 500+ , nan
   - This candidate has experience teaching English in Korea and is interested in pursuing a career in Human Resources.

3. Aspiring Human Resources Professional, Raleigh-Durham, North Carolina Area, 44, nan
   - This candidate is actively seeking a career in Human Resources and has a strong interest in the field.

4. People Development Coordinator at Ryan, Denton, Texas, 500+ , nan
   - This candidate has experience in people development and is looking for opportunities in Human Resources.

5. Advisory Board Member at Celal Bayar University, İzmir, Türkiye, 500+ , nan
   - This candidate has

In [27]:
response_2 = query_engine.query('Why did you chose the first candidate when there were other candidates that had the same job title?')

In [28]:
display_response(response_2)

**`Final Response:`** The selection was based on the specific requirements and preferences outlined in the query. The first candidate was chosen because they had a relevant educational background and were located in the desired area.

In [29]:
response_3 = query_engine.query('What do you mean by the deired area?')

In [30]:
display_response(response_3)

**`Final Response:`** The desired area refers to the geographical location where an individual is seeking employment opportunities. It is the specific region or city where they are interested in working and relocating to.

In [ ]:
response_4 = query_engine.query('But what if I am looking for people who are not in the desired area?')

[WARNING|tokenization_utils_base.py:3928] 2025-01-16 20:30:51,050 >> Token indices sequence length is longer than the specified maximum sequence length for this model (2058 > 2048). Running this sequence through the model will result in indexing errors


In [ ]:
display_response(response_4)

## Fine Tuning

In [ ]:
# MAX_SEQUENCE_LENGTH = 2048
# DATASET = load_dataset("huawei-noah/human_rank_eval", split='HumanRankEvalSoftEng')

# DATASET

In [ ]:
# train_dataset = DATASET.select(range(int(DATASET.shape[0]*.80)))
# train_dataset

In [ ]:
# ##################
# # Data Processing
# ##################

# # tokenizer = get_chat_template(tokenizer, chat_template = "phi-3", mapping={'role': 'text', 'content': 'votes', 'user': 'human', 'assistant': 'model'})

# def formatting_prompts_func(examples, tokenizer):
#   answers = examples['answers']
#   examples["text"] = [tokenizer.apply_chat_template(
#           answer, tokenize=False, add_generation_prompt=False) for answer in answers]
#   return examples

# train_dataset = train_dataset.map(
#     formatting_prompts_func,
#     fn_kwargs={"tokenizer": tokenizer},
#     batched=True)

# print(f"\ntrain_dataset features: {train_dataset.features}\n")

# train_dataset['text'][0]

In [ ]:
# ###########
# # Training
# ###########
# trainer = SFTTrainer(
#     model=model,
#     args=train_conf,
#     peft_config=peft_conf,
#     train_dataset=train_dataset,
#     # eval_dataset=processed_test_dataset,
#     # max_seq_length=MAX_SEQUENCE_LENGTH,
#     # dataset_text_field="text",
#     tokenizer=tokenizer,
#     # packing=True
# )
# train_result = trainer.train()
# metrics = train_result.metrics
# trainer.log_metrics("train", metrics)
# trainer.save_metrics("train", metrics)
# trainer.save_state()

In [ ]:
# #############
# # Evaluation
# #############
# tokenizer.padding_side = 'left'
# # metrics = trainer.evaluate()
# # metrics["eval_samples"] = len(processed_test_dataset)
# trainer.log_metrics("eval", metrics)
# trainer.save_metrics("eval", metrics)


# # ############
# # # Save model
# # ############
# trainer.save_model(train_conf.output_dir)

In [ ]:
# cwd = os.getcwd()
# model_path = cwd + '/drive/MyDrive/Phi-3FineTuning/checkpoint_dir'
# model = AutoModelForCausalLM.from_pretrained(model_path, **model_kwargs)

In [ ]:
# messages = [
#     {
#         "role": "user",
#         "content": "Given a phrase, answer the question by giving the top items, which will be given by the user."},
# ]

# model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
# input_length = model_inputs.shape[1]
# generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=500)

In [ ]:
# print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

In [ ]:
# model.save_pretrained('/drive/MyDrive/Phi-3FineTuning/post_finetuned_model')
# tokenizer.save_pretrained('/drive/MyDrive/Phi-3FineTuning/post_finetuned_model')

In [ ]:
# model = AutoModelForCausalLM.from_pretrained('/drive/MyDrive/Phi-3FineTuning/post_finetuned_model', **model_kwargs)
# tokenizer = AutoTokenizer.from_pretrained('/drive/MyDrive/Phi-3FineTuning/post_finetuned_model')

In [ ]:
# def generate_candidates(candidate_list, top_n_cands):
#   '''
#   --Parameters--
#   candidate_list: list of candidates
#   top_n_cands: number of candidates to return

#   --Returns--
#   list of top n candidates
#   '''
#   message = [
#       {"role": "user",
#        "content": f"Can you rank these candidates in this list {candidate_list} by the phrase 'aspiring human resources' and return the top {top_n_cands}?"},
#   ]


#   model_inputs = tokenizer.apply_chat_template(message, add_generation_prompt=True, return_tensors="pt")
#   input_length = model_inputs.shape[1]
#   with torch.no_grad():
#     outputs = model.generate(model_inputs, max_new_tokens=500)
#   response = tokenizer.batch_decode(outputs[:, input_length:], skip_special_tokens=True)[0]
#   return response

In [ ]:
# generate_candidates(candidates, 15)

## Unsloth

Run if you have access to a GPU

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template
import datasets
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
# import xformers

MAX_SEQUENCE_LENGTH = 2048
MODEL ="unsloth/Phi-3-mini-4k-instruct"
DATASET = load_dataset('huawei-noah/human_rank_eval', split='HumanRankEvalSoftEng')

model = AutoModelForCausalLM.from_pretrained(MODEL, trust_remote_code=True, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL,
    max_seq_length = MAX_SEQUENCE_LENGTH,
    dtype = None,
    load_in_4bit = False,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    max_seq_length = MAX_SEQUENCE_LENGTH,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
DATASET

In [ ]:
DATASET.select(range(int(DATASET.shape[0]*.80)))

In [ ]:
DATASET['answers'][0]

In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template = "phi-3", mapping={'role': 'text', 'content': 'votes', 'user': 'human', 'assistant': 'model'})

def formatting_prompts_func(examples, tokenizer):
  answers = examples['answers']
  examples["text"] = [tokenizer.apply_chat_template(
          answer, tokenize=False, add_generation_prompt=False) for answer in answers]
  return examples

DATASET = DATASET.map(
    formatting_prompts_func,
    fn_kwargs={"tokenizer": tokenizer},
    batched=True)

DATASET['text'][0]

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"

trainer = SFTTrainer(
    model = model,
    train_dataset = DATASET,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQUENCE_LENGTH,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407
    ),
)


In [ ]:
import wandb
trainer.train()

In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template = "phi-3", add_eos_token = True)

FastLanguageModel.for_inference(model)

messages = [
    {
        "role": "system",
        "content": "You are a ranking algorithm that ranks a list of items based on how similar each item is to a given phrase and returns the top 'n' items that the user requests."},
    {
        'role': 'user',
        'content': ''
    }
]

model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
# print(model_inputs.shape)
input_length = model_inputs.shape[1]
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=50)

In [ ]:
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])